# Modelo de cruce de calles

This notebook presents a tutorial for beginners on how to create a simple agent-based model with the [agentpy](https://agentpy.readthedocs.io) package. 
It demonstrates how to create a basic model with a custom agent type, run a simulation, record data, and visualize results.

In [1]:
# Model design
import agentpy as ap
import numpy as np 
import random
# Visualization
import seaborn as sns
import pandas as pd

## About the model

The model explores the distribution of wealth under a trading population of agents. 
Each agent starts with one unit of wealth. 
During each time-step, each agents with positive wealth 
randomly selects a trading partner and gives them one unit of their wealth.
We will see that this random interaction will create an inequality of wealth that 
follows a [Boltzmann distribution](http://www.phys.ufl.edu/~meisel/Boltzmann.pdf).
The original version of this model been written in [MESA](https://mesa.readthedocs.io/) 
and can be found [here](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html).

## Model definition

Se declara el agente carro

In [2]:
def abs(x):
    if x<0:
        x = x*-1
    return x

In [389]:
val = ["arriba","izquierda","abajo","derecha"]
arr = 0
izq = 0
aba = 0
der = 0
class CarAgent(ap.Agent):
    def setup(self):
        self.group = val[random.randint(0,3)]
        self.reaction = random.randint(10,20)
        global arr, izq, aba, der
        if(self.group=="arriba"):
            self.pos_X = 45
            self.pos_Y = 95 + arr * 15
            self.speed_X = 0
            self.speed_Y = -random.randint(30,50)
            arr += 1
        elif(self.group=="izquierda"):
            self.pos_X = 5 - izq * 15
            self.pos_Y = 45
            self.speed_X = random.randint(30,50)
            self.speed_Y = 0
            izq += 1
        elif(self.group=="abajo"):
            self.pos_X = 55
            self.pos_Y = 5 - aba * 15
            self.speed_X = 0
            self.speed_Y = random.randint(30,50)
            aba += 1
        elif(self.group=="derecha"):
            self.pos_X = 95 + der * 15
            self.pos_Y = 55
            self.speed_X = -random.randint(30,50)
            self.speed_Y = 0
            der += 1
    def move(self):
        res = True
        if(self.group=="arriba" and abs(65-self.pos_Y)<5 and 65>=self.pos_Y):
            res = self.model.semaforos[0].on
        elif(self.group=="izquierda" and abs(35-self.pos_X)<5 and 35>=self.pos_X):
            res = self.model.semaforos[1].on
        elif(self.group=="abajo" and abs(self.pos_Y-35)<5 and 35<=self.pos_Y):
            res = self.model.semaforos[2].on
        elif(self.group=="derecha" and abs(self.pos_X-65)<5 and 65<=self.pos_X):
            res = self.model.semaforos[3].on
        return res
    def carInfront(self):
        res = False
        if(self.group=="arriba"):
            infront = self.model.cars.select(self.model.cars.pos_Y < self.pos_Y)
            infront = infront.select(abs(infront.pos_Y - self.pos_Y) < self.reaction)
            infront = infront.select(infront.group == "arriba")
            if len(infront) > 0:
                res = True
        elif(self.group=="izquierda"):
            infront = self.model.cars.select(self.model.cars.pos_X > self.pos_X)
            infront = infront.select(infront.pos_X - self.pos_X < self.reaction)
            infront = infront.select(infront.group == "izquierda")
            if len(infront) > 0:
                res = True
        elif(self.group=="abajo"):
            infront = self.model.cars.select(self.model.cars.pos_Y > self.pos_Y)
            infront = infront.select(infront.pos_Y - self.pos_Y < self.reaction)
            infront = infront.select(infront.group == "abajo")
            if len(infront) > 0:
                res = True
        elif(self.group=="derecha"):
            infront = self.model.cars.select(self.model.cars.pos_X < self.pos_X)
            infront = infront.select(abs(infront.pos_X - self.pos_X)< self.reaction)
            infront = infront.select(infront.group == "derecha")
            if len(infront) > 0:
                res = True
        return res
    def drive(self):
        if CarAgent.move(self) and (not CarAgent.carInfront(self)):
            self.pos_X += self.speed_X*0.1
            self.pos_Y += self.speed_Y*0.1

In [390]:
class SemaforoAgent(ap.Agent):
    def setup(self):
        self.on = False
    def cambia(self):
        self.on = True

Esto es lo que se hace en el modelo:
- `setup` crea el arreglo de agentes
- `step` metodos que se llaman en cada iteracion
- `update` datos que se guardan en cada iteracion
- `end` llamado al finalizar el modelo

In [391]:
class StreetModel(ap.Model):

    def setup(self):
        self.cars = ap.AgentList(self, self.p.agents, CarAgent)
        self.semaforos = ap.AgentList(self, 4, SemaforoAgent)
        self.actual = 0
        self.semaforos[self.actual].on = True
        
    def step(self):
        self.cars.drive()

        if self.t % self.p.cambia == 0:
            self.semaforos[self.actual].on = False
            self.actual += 1
            if(self.actual>3):
                self.actual = 0
            self.semaforos[self.actual].on = True

    def update(self):
        self.cars.record('pos_X')
        self.cars.record('pos_Y')
        self.semaforos.record('on')

    def end(self):
        self.cars.record('pos_X')
        self.cars.record('pos_Y')
        print(self.cars.group)

## Simulation run

Parametros de una simulacion

In [392]:
parameters = {
    'agents': 20,
    'steps': 150,
    'seed': 42,
    'cambia': 10
}

Ejecucion de una simulacion

In [393]:
arr = 0
izq = 0
aba = 0
der = 0
model = StreetModel(parameters)
results = model.run()

Completed: 150 steps['arriba', 'izquierda', 'izquierda', 'derecha', 'derecha', 'arriba', 'abajo', 'abajo', 'izquierda', 'izquierda', 'abajo', 'arriba', 'derecha', 'derecha', 'izquierda', 'izquierda', 'abajo', 'derecha', 'izquierda', 'derecha']

Run time: 0:00:00.272279
Simulation finished


## Recopilacion de resultados

DataDict de la simulacion

In [394]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 4 keys
'variables': 
    'CarAgent': DataFrame with 2 variables and 3020 rows
    'SemaforoAgent': DataFrame with 1 variable and 604 rows
'reporters': DataFrame with 1 variable and 1 row
}

Informacion de la simulacion

In [395]:
results.info

{'model_type': 'StreetModel',
 'time_stamp': '2021-11-26 12:49:59',
 'agentpy_version': '0.1.4',
 'python_version': '3.9.0',
 'experiment': False,
 'completed': True,
 'created_objects': 24,
 'completed_steps': 150,
 'run_time': '0:00:00.272279'}

Tabla de posiciones para cada carro

In [396]:
results.variables.CarAgent

pos_X  pos_Y
obj_id t                
1      0     45.0   95.0
       1     45.0   91.0
       2     45.0   87.0
       3     45.0   83.0
       4     45.0   79.0
...           ...    ...
20     146 -248.6   55.0
       147 -253.2   55.0
       148 -253.2   55.0
       149 -257.8   55.0
       150 -262.4   55.0

[3020 rows x 2 columns]

Escribir tabla en csv

In [397]:
df = pd.DataFrame()
df = df.assign(id=None)
df = df.assign(t=None)
df = df.assign(X=None)
df = df.assign(Y=None)
id = []
t = []
x = []
y = []
for i in range(0,parameters["agents"]):
    for j in range(0,parameters["steps"]+1):
        id.append(i+1)
        t.append(j)
for i in results.variables.CarAgent.pos_X:
    x.append(i)
for i in results.variables.CarAgent.pos_Y:
    y.append(i)
df['id'] = id
df['t'] = t
df['X'] = x
df['Y'] = y
df.to_csv("carros.csv",index=False)

Tabla de valores de encendido para cada semaforo

In [398]:
results.variables.SemaforoAgent

on
obj_id t         
21     0     True
       1     True
       2     True
       3     True
       4     True
...           ...
24     146  False
       147  False
       148  False
       149  False
       150   True

[604 rows x 1 columns]

Escribir tabla en csv

In [399]:
df = pd.DataFrame()
df = df.assign(id=None)
df = df.assign(t=None)
df = df.assign(on=None)
id = []
t = []
on = []
for i in range(parameters["agents"],parameters["agents"]+4):
    for j in range(0,parameters["steps"]+1):
        id.append(i+1)
        t.append(j)
for i in results.variables.SemaforoAgent.on:
    on.append(i)
df['id'] = id
df['t'] = t
df['on'] = on
df.to_csv("semaforos.csv",index=False)